In [ ]:
#| default_exp routes.page

In [ ]:
#| export
import httpx
import pandas as pd

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda


In [ ]:
#| export
async def get_page_by_id(auth: dmda.DomoAuth, page_id: str,
                         debug_api: bool = False, 
                         session: httpx.AsyncClient = None,
                         ) -> rgd.ResponseGetData:
    url = f'https://{auth.domo_instance}.domo.com/api/content/v3/stacks/{page_id}/cards'

    res = await gd.get_data(
        auth=auth,
        url=url,
        method='GET',
        debug_api=debug_api,
        session = session,
    )

    res.response


    return res

#### sample get_page_by_id

In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

page_id = 1761849366
res = await get_page_by_id(page_id=page_id, auth=token_auth)

from pprint import pprint
pprint(res.response)


{'cards': [{'active': True,
            'allowTableDrill': True,
            'badgeUpdated': 1631619767000,
            'created': 1631619158,
            'creatorId': 1898323170,
            'description': '',
            'id': 1548948000,
            'locked': False,
            'metadata': {'SummaryNumberFormat': '{"type":"number","format":"#A"}',
                         'calendar': 'default',
                         'chartType': 'badge_vert_multibar',
                         'chartVersion': '8',
                         'columnAliases': '{}',
                         'columnFormats': '{}',
                         'currentLabel': '',
                         'currentMethod': 'empty',
                         'historyId': '6fa216c4-1465-4242-917c-3ca9e018c8be'},
            'ownerId': 1898323170,
            'title': 'View of 75th Percentile Test - Baseball Stats',
            'type': 'kpi',
            'urn': '1548948000'},
           {'active': True,
            'allowTableDril

In [ ]:
# | export
async def get_page_definition(auth, page_id, debug_api: bool = False, session: httpx.AsyncClient = None):

    url = f"https://{auth.domo_instance}.domo.com/api/content/v3/stacks/{page_id}/cards"

    params = {"includeV4PageLayouts": "true",
              "parts": "metadata,datasources,library,drillPathURNs,certification,owners,dateInfo,subscriptions,slicers"}

    res = await gd.get_data(url,
                            method='GET',
                            auth=auth,
                            session=session,
                            params=params, debug_api=debug_api)

    return res


#### sample implementation of get_page_definition

In [ ]:
from pprint import pprint
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

page_id = 1761849366
res = await get_page_definition(page_id=page_id, auth=token_auth)

pprint(res.response)


{'cards': [{'active': True,
            'allowTableDrill': True,
            'badgeUpdated': 1631619716000,
            'certification': {'adminCertified': False,
                              'state': 'NOT_CERTIFIED'},
            'created': 1631619693,
            'creatorId': 1898323170,
            'datasources': [{'adc': False,
                             'dataSourceId': '4ef43af5-67e7-4b9e-bd58-c4e592aa289a',
                             'dataSourceName': 'View of 75th Percentile Test - '
                                               'Baseball Stats',
                             'dataType': 'dataset-view',
                             'displayType': 'dataset-view',
                             'isSampleData': False,
                             'lastUpdated': 1673375782860,
                             'phase': None,
                             'providerType': 'dataset-view',
                             'state': 'IDLE'}],
            'dateInfo': {},
            'description'

In [ ]:
# | export
async def get_page_access_list(auth,
                              page_id,
                              is_expand_users: bool = True,
                              debug_api: bool = False, session: httpx.AsyncClient = None):
    """retrieves accesslist, which users and groups a page is shared with"""

    url = f"https://{auth.domo_instance}.domo.com/api/content/v1/share/accesslist/page/{page_id}?expandUsers={is_expand_users}"

    res = await gd.get_data(url,
                            method='GET',
                            auth=auth,
                            session=session, debug_api=debug_api)

    # add group members to users response
    if is_expand_users:
        group_users = [user for group in res.response.get(
            'groups') for user in group.get('users')]
        users = res.response.get('users') + group_users
        res.response.update({'users': users})

    return res


#### sample get_page_accesslist

In [ ]:
from pprint import pprint
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

page_id = 1761849366
res = await get_page_access_list(page_id=page_id, auth=token_auth, is_expand_users=True)

# for user in res.response.get('users'):
#     print({user.get('displayName')})

pprint(res.response)

{'groups': [{'active': True,
             'creatorId': 1893952720,
             'default': False,
             'description': 'updated via upsert_group on 2023-04-19',
             'hidden': False,
             'id': 1944377875,
             'name': 'Test Group 2q32',
             'type': 'closed',
             'users': [{'accepted': False,
                        'active': True,
                        'anonymous': True,
                        'created': 1618299284,
                        'displayName': 'Adam Landefeld ',
                        'emailAddress': 'Adam.Landefeld@domo.com',
                        'id': 1542225148,
                        'invitorUserId': 1893952720,
                        'modified': 1665166064000,
                        'pending': True,
                        'roleId': 2097317660,
                        'systemUser': False,
                        'userName': 'Adam.Landefeld@domo.com',
                        'userType': 'USER'},
                

In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()